# Creating Designs by Leveraging OpenAI and Gradio UI

## Overview
In this project, we explore how to leverage OpenAI's API (specifically DALL-E) and Gradio's UI
to create a platform that generates bespoke images from user-provided text prompts.

---

## 1. Import Essential Libraries

We need the following libraries:
- openai: for accessing OpenAI's API (DALL-E),
- requests: for making HTTP requests to retrieve generated images,
- PIL (from Pillow): for manipulating and displaying images,
- io.BytesIO: for handling binary data (image files in memory),
- gradio: for building a user-friendly web UI.

In [ ]:
# !pip install openai gradio  # Uncomment if you need to install in a fresh environment

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
from PIL import Image
from io import BytesIO
import logging
import traceback

In [2]:
# Set up logging for debugging
logging.basicConfig(level=logging.DEBUG)

---

## 2. Configure OpenAI API Key

Your API key is required for authenticating requests to OpenAI.

In [ ]:
load_dotenv(verbose=True, dotenv_path='.env', override=True)

# Retrieve the API key from the environment
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("API key not found. Please ensure 'OPENAI_API_KEY' is set in your .env file.")

# Create the OpenAI client with the API key

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def test_openai_api():
    try:
        # Create a chat completion
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": "Say this is a test"},
            ],
        )
        # Print the response content
        print(response.choices[0].message.content)
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the test function
test_openai_api()

---

## 3. Define the Image Generation Function

This function takes a text prompt, calls DALL-E to generate an image, 
retrieves the image from the returned URL, and returns it as a Pillow Image object.

In [ ]:
import requests
from PIL import Image
from io import BytesIO

def generate_image(prompt):
    """
    Generates an image based on the specified text prompt using OpenAI's image generation API.

    Args:
        prompt (str): The text prompt describing the desired image.

    Returns:
        PIL.Image: A PIL image generated from the prompt.
    """
    try:
        # Validate the prompt input
        if not prompt.strip():
            raise ValueError("Prompt cannot be empty.")

        # Call OpenAI to create the image
        response = client.images.generate(
            model='dall-e-3',
            prompt=prompt,
            size="512x512",
            quality="standard",
            n=1,
        )

        # Extract the image URL from the response
        image_url = response.data[0].url
        # print(f"Image URL: {image_url}")

        # Fetch the image from the URL
        image_response = requests.get(image_url)
        image_response.raise_for_status()  # Raise an error for bad status codes

        # Convert the image to a PIL object and return
        img = Image.open(BytesIO(image_response.content))
        return img

    except Exception as e:
        # Log the error and return a placeholder image
        print(f"Error: {e}")
        return create_error_image(str(e))


def create_error_image(message):
    """
    Creates a placeholder image with an error message.
    """
    from PIL import ImageDraw

    img = Image.new('RGB', (512, 512), color=(255, 0, 0))  # Red background
    draw = ImageDraw.Draw(img)
    draw.text((10, 10), f"Error: {message}", fill=(255, 255, 255))  # White text
    return img

In [ ]:
# prompt="A surreal landscape with floating islands and a pink sky"
# img = generate_image(prompt)
# display(img)

---

## 4. Create the Gradio Interface

We'll set up a Gradio interface with:
- A text input box for the user prompt
- An image output to display the generated image

In [ ]:
# Set up the Gradio interface
interface = gr.Interface(
    fn=generate_image,     # The function that handles generation
    inputs="text",         # We'll take a simple text string as input
    outputs="image",       # We'll return a generated image
    title="AI-Generated Designs for Netflix Campaigns",
    description=(
        "Enter a detailed text prompt describing the design you'd like to see. "
        "For instance, 'a futuristic cityscape with neon lights, in the style of cyberpunk' "
        "or 'a minimalist poster featuring a detective theme for a new Netflix series.'"
    )
)

---

## 5. Launch the Gradio Interface

Running `launch()` will start a local web server (and display a shareable link in some environments).
Clicking the link will open the Gradio interface where you can enter prompts and see generated images.

In [ ]:
interface.launch()

---

## Result

By completing this notebook, I have illustrated the powerful synergy between OpenAI’s DALL-E
and Gradio’s UI. This platform enables designers to quickly prototype and generate custom 
visual content for high-profile marketing campaigns, such as those by Netflix.